<a href="https://colab.research.google.com/github/sherna90/analisis_algoritmos/blob/master/or_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP OR-Tools

In [12]:
import pandas as pd

def load_nodes(path):
    df=pd.read_csv(path)
    df.rename({'X':'lon','Y':'lat'},axis='columns',inplace=True)
    return df

def load_data(path):
    df=pd.read_csv(path, encoding = 'utf-8',dtype={'InputID':'string','TargetID':'string','Distance':float}) 
    df['InputID'] = df['InputID'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
    df['TargetID'] = df['TargetID'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
    df['TargetID']=df['TargetID'].astype('category')
    df['InputID']=df['InputID'].astype('category')
    return df

In [13]:
dist_path='https://raw.githubusercontent.com/sherna90/analisis_algoritmos/master/tsp_demo/data/distancias_maule.csv'
edge_data=load_data(dist_path)
nodes_path='https://raw.githubusercontent.com/sherna90/analisis_algoritmos/master/tsp_demo/data/comunas_maule.csv'
node_data=load_nodes(nodes_path)

In [135]:
from scipy.spatial.distance import pdist,squareform

dist_mat=squareform(pdist(node_data[['lon','lat']].values,metric='euclidean')).tolist()

In [138]:
get_distance = lambda input,target : float(edge_data[(edge_data.InputID==input) & (edge_data.TargetID==target)]['Distance'].values/1000)

In [139]:
dist_mat=[ [0 for target in edge_data.TargetID.unique()] for input in edge_data.InputID.unique()]

In [140]:
for i,input in enumerate(edge_data.InputID.unique()):
  for j,target in enumerate(edge_data.InputID.unique()):
    if input!=target:
      dist_mat[i][j]=int(get_distance(input,target))
    

# OR Tools


In [126]:
!pip install ortools

In [127]:
def create_data_model(dist_mat):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist_mat
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [128]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Objective: {}m\n'.format(route_distance)

In [141]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


# Create routing model
if tsp_size > 0:
  data = create_data_model(dist_mat)
  # Create the routing index manager.
  manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                         data['num_vehicles'], data['depot'])

  # Create Routing Model.
  routing = pywrapcp.RoutingModel(manager)
  def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()

  def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  # Define cost of each arc.
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
  search_parameters.time_limit.seconds = 30
  search_parameters.log_search = True
  search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
  solution = routing.SolveWithParameters(search_parameters)

  # Print solution on console.
  if solution:
      print_solution(manager, routing, solution)

Objective: 781
Route:
 0 -> 9 -> 10 -> 25 -> 19 -> 26 -> 27 -> 28 -> 21 -> 22 -> 24 -> 23 -> 3 -> 4 -> 29 -> 20 -> 11 -> 7 -> 6 -> 1 -> 8 -> 14 -> 15 -> 16 -> 17 -> 5 -> 13 -> 12 -> 18 -> 2 -> 0

